In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.preprocessing import image
from keras import applications
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)


training_set = datagen.flow_from_directory(
    '/content/drive/MyDrive/Food_Crops/train',
        target_size=(224, 224),
        batch_size=32,
        shuffle= False,
        class_mode='categorical')

val_set = datagen.flow_from_directory(
        '/content/drive/MyDrive/Food_Crops/validate',
        target_size=(224, 224),
        batch_size=16,
        shuffle= False,
        class_mode="categorical")

test_set = datagen.flow_from_directory(
        '/content/drive/MyDrive/Food_Crops/test',
        target_size=(224, 224),
        batch_size=1,
        shuffle= False,
        class_mode="categorical")

In [ ]:
csv_logger = tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/Results/DenseNet201_training.csv', append=True)

model_DenseNet201 = tf.keras.applications.DenseNet201(input_shape=(224, 224,3), weights='imagenet', include_top=False)

for layer in model_DenseNet201.layers:
    layer.trainable = False

x = Flatten()(model_DenseNet201.layers[-1].output)

output= Dense(12, activation = 'softmax')(x)

model = Model(inputs = model_DenseNet201.inputs, outputs = output)

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

r1 = model.fit(training_set,epochs=50,validation_data = val_set, callbacks = [csv_logger])

Y_pred = model.predict(test_set)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
cm = confusion_matrix(test_set.classes, y_pred)
df = pd.DataFrame(cm, index=None)
df.to_excel("/content/drive/MyDrive/Results/DenseNet201_confusionmatrix.xlsx")
print(cm)

print('Classification Report')
print(classification_report(test_set.classes, y_pred))
report = classification_report(test_set.classes, y_pred, output_dict=True)
dff = pd.DataFrame(report).transpose()
dff.to_excel("/content/drive/MyDrive/Results/DenseNet201_classificationreport.xlsx")